<a href="https://colab.research.google.com/github/Isma7471/Book-Recommendation-Engine-using-KNN/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Instructions
In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the train_dataset and 20% of the data as the test_dataset.

pop off the "expenses" column from these datasets to create new datasets called train_labels and test_labels. Use these labels when training your model.

Create a model and train it with the train_dataset. Run the final cell in this notebook to check your model. The final cell will use the unseen test_dataset to check how well the model generalizes.

To pass the challenge, model.evaluate must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the test_dataset and graph the results.

# **Importing** **Libraries**

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

# **Loading** **Dataset**

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

# Checking for Missing Values

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.dtypes

In [ ]:
dataset.region.value_counts()

In [ ]:
dataset = dataset.join(pd.get_dummies(dataset.region, prefix='region')).drop('region', axis=1)
dataset.head()

In [ ]:
dataset.sex.value_counts()

In [ ]:
labels = {}

values        = dataset.sex.astype('category')
labels['sex'] = values.cat.categories
dataset['sex']     = values.cat.codes

dataset.head()

In [ ]:
dataset.smoker.value_counts()

In [ ]:
values           = dataset.smoker.astype('category')
labels['smoker'] = values.cat.categories
dataset['smoker']     = values.cat.codes

dataset.head()

In [ ]:
labels

In [ ]:
import seaborn as sns
sns.heatmap(dataset.corr(), annot=True, fmt='.2f')

In [ ]:
dataset.drop(['region_northeast', 'region_northwest', 'region_southeast', 'region_southwest'], axis=1, inplace=True)


In [ ]:
dataset.drop(['sex', 'children'], axis=1, inplace=True)

In [ ]:
sns.pairplot(dataset)

In [ ]:
dataset=dataset.sample(frac=1)

In [ ]:
size = int(len(dataset) * .2)
train_dataset = dataset[:-size]
test_dataset = dataset[-size:]
print(len(dataset), len(train_dataset), len(test_dataset))

In [ ]:
train_labels = train_dataset['expenses']
train_dataset = train_dataset.drop('expenses', axis=1)

In [ ]:
test_labels = test_dataset['expenses']
test_dataset = test_dataset.drop('expenses', axis=1)

In [ ]:
model=keras.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mae', metrics=['mae', 'mse'])
model.summary()

In [ ]:
class EpochDots(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0:
      print()
      print('Epoch: {:d}, '.format(epoch), end='')
      for name, value in sorted(logs.items()):
        print('{}:{:0.4f}'.format(name, value), end=', ')
        print()

    print('.', end='')

In [ ]:
r=model.fit(train_dataset, train_labels, epochs=500,
          verbose=0, callbacks=[EpochDots()])

In [ ]:
res=model.evaluate(test_dataset, test_labels, verbose=2)
print(res)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
